# The Game of Life

In this notebook, we want to simulate the Game of Life

## A few informations about JuPyteR

### You can use maths in the LaTeX way

Math in text $\alpha = 5$ is possible as well as standalone:

$$
\Psi = \int_{0}^{\infty} \omega(t)dt
$$


### You can use many environments

* list
* list


1. numbered list
2. tada


### Stuff from Internet

![](https://upload.wikimedia.org/wikipedia/commons/d/d8/Game_of_life_U.gif)


## Game of Life

Documentation about GoL is available [here on wikipedia](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life)

The rule of the game is:

* Cells can be either dead (0) or alive (1)
* At every step, there is are two rules:
    * A birth rule: a dead cell becomes alive if it has 3 living neigbors
    * A survival rule! a living cell stays alive if it has 2 or 3 living neighbors
    
The rule can be summarized as `B3S23`    

In [17]:
# Matplotlib "notebook" backend is an interactive backend for notebooks
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

## Let's code the GoL

### First, we need to compute the Moore neighbors

![](https://upload.wikimedia.org/wikipedia/commons/4/4d/Moore_neighborhood_with_cardinal_directions.svg)

In [20]:
# Starting cells
cells = np.random.randint(2, size=(10, 10))
cells

array([[1, 1, 0, 1, 0, 1, 0, 1, 1, 0],
       [0, 1, 0, 0, 1, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 1, 0, 0, 0, 1, 0, 1, 1],
       [0, 0, 1, 1, 0, 0, 1, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 1, 0, 1, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 1, 0, 0, 1],
       [0, 0, 1, 0, 1, 1, 1, 1, 1, 1],
       [0, 1, 0, 1, 1, 1, 0, 0, 0, 1]])

In [22]:
cells[0] # First line

array([1, 1, 0, 1, 0, 1, 0, 1, 1, 0])

In [25]:
cells[9] # Last line

array([0, 1, 0, 1, 1, 1, 0, 0, 0, 1])

In [26]:
cells[-1] # Also the last line

array([0, 1, 0, 1, 1, 1, 0, 0, 0, 1])

In [27]:
cells[5,6]

1

In [28]:
cells[1:5]

array([[0, 1, 0, 0, 1, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 1, 0, 0, 0, 1, 0, 1, 1],
       [0, 0, 1, 1, 0, 0, 1, 1, 0, 0]])

In [29]:
cells[3:7,4:8]

array([[0, 0, 1, 0],
       [0, 0, 1, 1],
       [0, 1, 1, 0],
       [0, 0, 0, 0]])

In [30]:
cells[:5, :-2]

array([[1, 1, 0, 1, 0, 1, 0, 1],
       [0, 1, 0, 0, 1, 1, 0, 1],
       [0, 1, 0, 0, 0, 1, 1, 1],
       [0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 1, 1, 0, 0, 1, 1]])

In [34]:
# Count the cells
Nr, Nc = cells.shape # Number of rows and columns
for r in range(Nr): # Loop on row
    for c in range(Nc) : # Loop over columns
        cell = cells[r, c]
        




In [35]:
cells

array([[1, 1, 0, 1, 0, 1, 0, 1, 1, 0],
       [0, 1, 0, 0, 1, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 1, 0, 0, 0, 1, 0, 1, 1],
       [0, 0, 1, 1, 0, 0, 1, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 1, 0, 1, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 1, 0, 0, 1],
       [0, 0, 1, 0, 1, 1, 1, 1, 1, 1],
       [0, 1, 0, 1, 1, 1, 0, 0, 0, 1]])

In [46]:
r= 9
c= 1
cells[max(0,r-1) : min(r+2, Nr), c-1:c+2]

array([[0, 0, 1],
       [0, 1, 0]])

In [47]:
%timeit max(1,2)

207 ns ± 25.3 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [52]:
from numpy import max as npmax
%timeit npmax([1,2])

9.98 µs ± 211 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [53]:
%prun npmax([1,2])